In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
import numpy as np
import time

# Hyperparameters
IMAGE_SIZE = (150,150)
EPOCHS = 100
BATCH_TRAIN_SIZE = 16
BATCH_SIZE = 15
INITIAL_LEARNING_RATE = 6.3e-5
L2_REGULARIZE = 1e-3

def image_gen_w_aug(train_parent_directory, validate_parent_directory, test_parent_directory):
    train_datagen = ImageDataGenerator(
        rescale=1/255,
        rotation_range=40,
        zoom_range=0.2,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(rescale=1/255)
    val_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(
        train_parent_directory,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_TRAIN_SIZE,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = val_datagen.flow_from_directory(
        validate_parent_directory,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = test_datagen.flow_from_directory(
        test_parent_directory,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator, test_generator

def model_output():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150,150, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(BatchNormalization())
    model.add(Flatten())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Dense(3, activation='softmax', kernel_regularizer=l2(L2_REGULARIZE)))

    return model

train_dir = os.path.join('./datasets/train')
test_dir = os.path.join('./datasets/test')
val_dir = os.path.join('./datasets/validation')

train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, val_dir, test_dir)

model = model_output()
model.compile(
    optimizer=Adam(learning_rate=INITIAL_LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // BATCH_TRAIN_SIZE


# Learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

# Start time
start_time = time.time()

model.summary()

with tf.device("/GPU:0"):
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator) + 1,
        epochs=EPOCHS,
        verbose=1,
        validation_data=validation_generator,
        callbacks=[lr_scheduler, early_stopping]
    )


# end Time
end_time = time.time()
# duration 
duration = end_time - start_time

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

tf.keras.models.save_model(model,'my_model.h5')

In [ ]:

# Plot training and validation accuracy
plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Model Accuracy")
plt.grid(axis='y')
plt.legend()

# Plot training and validation loss
plt.subplot(122)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Model Accuracy")
plt.legend()
plt.grid(axis='y')
plt.show()

# Convert duration to minutes and seconds
duration_min, duration_sec = divmod(duration, 60)
print(f'Total time taken to train the model: {int(duration_min)} minutes and {int(duration_sec)} seconds')
print()
# Print out the final accuracy values
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print()
# Print out the final loss values
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print()
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')